# 네이버 맵 크롤링
- 메인 코드에 샵(#)으로 변경해야 할 부분 표시해뒀습니다.
- 그리고 저장할때 이름만 변경하시면 됩니다.

-> 따라서 변경해야하는거 저장할때 이름과 인덱스 두개 입니다!!!!!

몇번 정도 남았는지 확인할 수 있음..!

In [ ]:

import pandas as pd

dataset_raw = pd.read_csv('구석구석dataset_수정.csv', encoding='cp949')
dataset = dataset_raw.copy()
dataset = dataset[['attraction']][0:4]
dataset_list = dataset['attraction'].to_list()
dataset_list

FileNotFoundError: [Errno 2] No such file or directory: '구석구석dataset_수정.csv'

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd

dataset_raw = pd.read_csv('구석구석dataset_수정.csv', encoding='cp949')
dataset = dataset_raw.copy()

#################################################################
# 변경해야 할 부분 (인덱스 부분만 변경 하시면 됩니다!)
dataset = dataset[['attraction']][0:4]
# 변경하실 때 인덱스만 변경하시면 됩니다!
################################################################
# 데이터 셋을 리스트로 변경(성공 실패 분리할때 사용될 예정)
dataset_list = dataset['attraction'].to_list()

# DataFrame of locations
# locations_df = dataset

# 웹 드라이버 설정
driver = webdriver.Chrome()
driver.maximize_window()

# 추출한 데이터 저장할 저장소 초기화
tourist_spot_list = []
good_point_list = []
selected_people_list = []


# 데이터 추출 코드
def extract_data():
    try:
        tourist_spot = spot

        # 사용 가능한 좋은 점 개수 가져오기
        good_points_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[1]/div/div/div[2]/ul/li')
        num_good_points = len(good_points_elements)

        for j in range(1, num_good_points + 1):
            try:
                # 좋은 점과 선택한 사람 추출
                good_point_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[1]/div/div/div[2]/ul/li[{j}]/div[2]/span[1]'
                good_point = driver.find_element(By.XPATH, good_point_xpath).text
                selected_people_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[1]/div/div/div[2]/ul/li[{j}]/div[2]/span[2]'
                selected_people = driver.find_element(By.XPATH, selected_people_xpath).text

                good_point_list.append(good_point)
                selected_people_list.append(selected_people)
                tourist_spot_list.append(tourist_spot)
            except Exception as e:
                print(f"항목 {j}은(는) 존재하지 않습니다. 오류: {e}")
                if 'driver' in locals():
                    driver.quit()
                return None

    except Exception as e:
        print(f"데이터 추출 중 오류 발생: {e}")

# 리뷰 버튼 클릭을 위한 코드
# 위 구석구석 데이터 샛의 attraction을 가져와 관광지명을 spot으로 추출
for index, row in dataset.iterrows():
    spot = row['attraction']
    driver.get(f"https://m.search.naver.com/search.naver?sm=mtp_sly.hst&where=m&query={spot}")
    time.sleep(10)

    # 네이버 검색창 화면에서 클릭 요소 선택
    xpaths = [
        '//*[@id="_title"]/a/span[1]',
        '//*[@id="place-main-section-root"]/div/div[5]/ul/li[1]/div[1]/div[2]/a/div[1]/div/span[1]',
        '//*[@id="place-main-section-root"]/div/div[4]/ul/li[1]/div[1]/div[2]/a/div[1]/div/span[1]',
        '//*[@id="loc-main-section-root"]/div/div[4]/ul/li[1]/div[1]/div[2]/a[1]/div/div/span[1]'
    ]

    element_clicked = False
    for xpath in xpaths:
        try:
            element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, xpath))
            )
            element.click()
            element_clicked = True
            break
        except TimeoutException:
            print(f"{spot}에 대한 첫 번째 검색 결과를 찾을 수 없습니다. XPath: {xpath}")


    if not element_clicked:
        continue

    # 리뷰 버튼 클릭
    review_xpaths = [
        '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[3]/span',
        '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[2]/span',
        '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[4]/span'
    ]

    try:
        for review_path in review_xpaths:
            review_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, review_path))
            )
            review_button.click()
            # 이런점이 좋았어요 다음 보기 8번 클릭
            button_class_name = "dP0sq"
            for i in range(8):
                try:
                    first_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CLASS_NAME, button_class_name))
                    )
                    driver.execute_script("arguments[0].click();", first_button)
                    print(f"{spot} - {i+1}번째 버튼 클릭 성공")
                    # 다음 보기 버튼 누르는 것에 성공한다면 원하는 데이터 추출
                    extract_data()

                except TimeoutException:
                    print(f"{spot} - {i+1}번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.")
                    break

    except TimeoutException:
        print(F"{spot}에 대한 리뷰 버튼을 찾지 못했습니다.")
        continue

# 추출한 데이터 데이터 프레임 형식으로 만들기
data = {
    'tourist_spot': tourist_spot_list,
    'good_point': good_point_list,
    'selected_people': selected_people_list
}
df = pd.DataFrame(data)
df = df.drop_duplicates()

# 실패 데이터와 성공 데이터 분리하기
def process_multiple_places(dataset_list,df):
    combined_df = df
    failed_places = []

    # dataset_list의 각 장소에 대해 final_df에 존재하지 않는 장소를 실패한 장소로 간주
    for place in dataset_list:
        if place not in combined_df['tourist_spot'].unique():
            failed_places.append({'place': place, 'error': '데이터 없음'})

    fail_df = pd.DataFrame(failed_places)
    print("모든 장소에 대한 데이터 프레임 결합 완료")

    return combined_df, fail_df

final_df, fail_df = process_multiple_places(dataset_list, df)
# 결과 출력
print("최종 데이터프레임:")
print(final_df)

print("실패한 장소 데이터프레임:")
print(fail_df)

# 드라이버 종료
driver.quit()

동대문역사문화공원 - 1번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
동대문역사문화공원 - 1번째 버튼 클릭 성공
동대문역사문화공원 - 2번째 버튼 클릭 성공
동대문역사문화공원 - 3번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
동대문역사문화공원 - 1번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
서울식물원에 대한 첫 번째 검색 결과를 찾을 수 없습니다. XPath: //*[@id="_title"]/a/span[1]
서울식물원에 대한 첫 번째 검색 결과를 찾을 수 없습니다. XPath: //*[@id="place-main-section-root"]/div/div[5]/ul/li[1]/div[1]/div[2]/a/div[1]/div/span[1]
서울식물원 - 1번째 버튼 클릭 성공
서울식물원 - 2번째 버튼 클릭 성공
서울식물원 - 3번째 버튼 클릭 성공
서울식물원 - 4번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
서울식물원 - 1번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
서울식물원 - 1번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
동대문디자인플라자(DDP)에 대한 첫 번째 검색 결과를 찾을 수 없습니다. XPath: //*[@id="_title"]/a/span[1]
동대문디자인플라자(DDP)에 대한 첫 번째 검색 결과를 찾을 수 없습니다. XPath: //*[@id="place-main-section-root"]/div/div[5]/ul/li[1]/div[1]/div[2]/a/div[1]/div/span[1]
동대문디자인플라자(DDP)에 대한 첫 번째 검색 결과를 찾을 수 없습니다. XPath: //*[@id="place-main-section-root"]/div/div[4]/ul/li[1]/div[1]/div[2]/a/div[1]/div/span[1]
동대문디자인플라자(DDP) - 1번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
동대문디자인플라자(DDP) - 1번째 버튼이 존재하지 않아 클릭하지 않고

In [ ]:
# 성공 데이터 확인
final_df

,tourist_spot,good_point,selected_people
0,동대문역사문화공원,"""뷰가 좋아요""",이 키워드를 선택한 인원\n22
1,동대문역사문화공원,"""관리가 잘 되어있어요""",이 키워드를 선택한 인원\n21
2,동대문역사문화공원,"""볼거리가 많아요""",이 키워드를 선택한 인원\n14
3,동대문역사문화공원,"""사진이 잘 나와요""",이 키워드를 선택한 인원\n12
4,동대문역사문화공원,"""산책로가 잘 되어있어요""",이 키워드를 선택한 인원\n12
5,동대문역사문화공원,"""편의시설이 잘 되어있어요""",이 키워드를 선택한 인원\n9
6,동대문역사문화공원,"""먹거리가 풍부해요""",이 키워드를 선택한 인원\n7
7,동대문역사문화공원,"""가격이 합리적이에요""",이 키워드를 선택한 인원\n7
8,동대문역사문화공원,"""방문객이 많아요""",이 키워드를 선택한 인원\n5
9,동대문역사문화공원,"""체험 프로그램이 다양해요""",이 키워드를 선택한 인원\n5


In [ ]:
# 성공 데이터 저장
final_df.to_csv('nn~nn성공데이터.csv')


In [ ]:
# 실패 데이터 확인
fail_df

,place,error
0,동대문디자인플라자(DDP),데이터 없음


In [ ]:
# 실패 데이터 저장
fail_df.to_csv('nn~nn실패데이터.csv')